# An example of dynamically growing and shrinking an Analytic partition to handle larger workloads.

This example assume that Moab/Tourge (PBS) is running and we have a ccm queue to handle allocations. In the fur

In [1]:
!yarn node -list

15/12/10 14:25:04 INFO client.RMProxy: Connecting to ResourceManager at nid00379/10.128.1.126:8032
Total Nodes:4
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
  nid00382:35733	        RUNNING	    nid00382:8042	                           2
  nid00380:40375	        RUNNING	    nid00380:8042	                           0
  nid00383:38910	        RUNNING	    nid00383:8042	                           1
  nid00381:52626	        RUNNING	    nid00381:8042	                           0


In [2]:
!hadoop fs -du -h 

90.9 M   .flink
178.3 M  .sparkStaging
610.4 M  HiBench
335.5 K  artofwar.txt
558.8 G  bigdata
191.6 K  hamlet.txt
18.4 M   ints.parquet
68.1 M   pagecounts
3.0 K    passwd
19.1 M   text8_linessmall


In [3]:
!qstat

We have a simple PBS/Moab script which dynamically increases the "pool" size, then after a predetermined time will bing the pool back to the default size (4) or the user can send a signal to the batch jobs to kill the session and revert it back to the default 4 nodes.

In [19]:
!(cd /home/users/jsparks/tmp ; rm -f jsparks-test.o*; qsub -q ccm_queue -l nodes=12:ppn=1 job.sh; sleep 2; qstat)

26052.sdb
Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
26052.sdb                  jsparks-test     jsparks                0 R ccm_queue      


In [20]:
!sleep 10; qstat -u ${USER}; yarn node -list

15/12/10 13:14:36 INFO client.RMProxy: Connecting to ResourceManager at nid00379/10.128.1.126:8032
Total Nodes:16
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
  nid00016:24361	        RUNNING	    nid00016:8042	                           0
  nid00007:14786	        RUNNING	    nid00007:8042	                           0
  nid00005:10089	        RUNNING	    nid00005:8042	                           0
  nid00014:23136	        RUNNING	    nid00014:8042	                           0
  nid00382:35733	        RUNNING	    nid00382:8042	                           0
  nid00006:11929	        RUNNING	    nid00006:8042	                           0
  nid00380:40375	        RUNNING	    nid00380:8042	                           2
  nid00383:38910	        RUNNING	    nid00383:8042	                           0
  nid00015:25506	        RUNNING	    nid00015:8042	                           0
  nid00008:13215	        RUNNING	    nid00008:8042	                           0
  nid0

To sneak peak at what your job is doing you can tail/cat you job file in /var/spool/torque/spool

In [2]:
!hadoop fs -rm -R -skipTrash /tmp/*.parquet
!hadoop fs -rm -R -skipTrash /tmp/*.seq

rm: `/tmp/*.parquet': No such file or directory
rm: `/tmp/*.seq': No such file or directory


Once you have more nodes, you can run larger hadoop/spark jobs.

**Note:** This is python3 so some functions have been renamed/depricated etc ... Just an FYI...

In [5]:
import random
import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext

sc.defaultParallelism

# Set the number of executor cores to match the system's allocation
data = sc.parallelize(range(1000)).flatMap(lambda x: [Row(a=random.randint(1, 10), num=random.randint(1, 100), str=("a" * random.randint(1, 30))) for i in range(10000)])
dataTable = sqlContext.createDataFrame(data)
# Depricated call
#dataTable.saveAsParquetFile("/tmp/ints1.parquet")
dataTable.write.parquet("/tmp/ints1.parquet")

data = sc.parallelize(range(1000)).flatMap(lambda x: [Row(a=random.randint(1, 10), num=random.randint(1, 100)) for i in range(10000)])
dataTable = sqlContext.createDataFrame(data)
# Depricated call
#dataTable.saveAsParquetFile("/tmp/ints2.parquet")
dataTable.write.parquet("/tmp/ints2.parquet")

data = sc.parallelize(range(1000)).flatMap(lambda x: [(random.randint(1, 10), random.randint(1, 100)) for i in range(10000)])
data.saveAsNewAPIHadoopFile('hdfs://nid00379:8020/tmp/ints3.seq',
                         'org.apache.hadoop.mapreduce.lib.output.SequenceFileOutputFormat', # Output format class
                         conf = {
                              'mapreduce.output.fileoutputformat.compress'       : 'true',
                              'mapreduce.output.fileoutputformat.compress.codec' : 'org.apache.hadoop.io.compress.DefaultCodec',
                              'mapreduce.output.fileoutputformat.compress.type'  : 'BLOCK'
                              }
                        )

In [7]:
!hadoop fs -ls /tmp
!hadoop fs -du -h  /tmp/ints1.parquet

Found 6 items
drwx------   - jsparks supergroup          0 2015-10-27 15:06 /tmp/hadoop-yarn
drwx-wx-wx   - jsparks supergroup          0 2015-11-20 13:15 /tmp/hive
drwxr-xr-x   - jsparks supergroup          0 2015-12-10 10:11 /tmp/ints1.parquet
drwxr-xr-x   - jsparks supergroup          0 2015-12-10 10:11 /tmp/ints2.parquet
drwxr-xr-x   - jsparks supergroup          0 2015-12-10 10:11 /tmp/ints3.seq
drwxrwxrwx   - jsparks supergroup          0 2015-11-18 11:37 /tmp/twitter
0      /tmp/ints1.parquet/_SUCCESS
340    /tmp/ints1.parquet/_common_metadata
1.1 K  /tmp/ints1.parquet/_metadata
9.2 M  /tmp/ints1.parquet/part-r-00000-cd276803-b726-401d-901c-a1583bf15c21.gz.parquet
9.2 M  /tmp/ints1.parquet/part-r-00001-cd276803-b726-401d-901c-a1583bf15c21.gz.parquet


In [21]:
!yarn node -list

15/12/10 13:35:51 INFO client.RMProxy: Connecting to ResourceManager at nid00379/10.128.1.126:8032
Total Nodes:4
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
  nid00382:35733	        RUNNING	    nid00382:8042	                           0
  nid00380:40375	        RUNNING	    nid00380:8042	                           2
  nid00383:38910	        RUNNING	    nid00383:8042	                           0
  nid00381:52626	        RUNNING	    nid00381:8042	                           4


#### Run the tests
See http://0x0fff.com/spark-dataframes-are-faster-arent-they/
for a better writeup.

In [19]:
import random
import time
from pyspark.sql import *
from pyspark.sql.functions import *
 
def mappart(part):
    counters = dict()
    for x,y in part:
        if not x in counters:
            counters[x] = [0, 0]
        counters[x][0] += 1
        counters[x][1] += y
    return counters.iteritems()
 
def fn():
    # Test 1
    runtime = 0.0
    for i in range(5):
        start = time.time()
        data = sqlContext.read.load("/tmp/ints1.parquet")
        #data = sqlContext.load("/tmp/ints1.parquet")
        data.groupBy("a").agg(col("a"), avg("num")).collect()
        runtime += time.time() - start
    print ("Test #1 Runtime: %7.2f sec" % (runtime / 5.0))
    # Test 2
    runtime = 0.0
    for i in range(5):
        start = time.time()
        pdata = sqlContext.read.load("/tmp/ints2.parquet")
        #pdata = sqlContext.load("/tmp/ints2.parquet")
        sum_count = pdata.map(lambda x: (x.a, [x.num, 1])).reduceByKey(lambda x, y:[x[0] + y[0], x[1] + y[1]]).collect()
        sum_count = [(x[0], float(x[1][0]) / x[1][1]) for x in sum_count]
        runtime += time.time() - start
    print ("Test #2 Runtime: %7.2f sec" % (runtime / 5.0))
 
fn()

Test #1 Runtime:    5.30 sec
Test #2 Runtime:   24.64 sec
